# 08 - External Index Retrievers 🌐

## Learning Objectives 🎯

In this notebook, you'll learn:

1. **What are External Index Retrievers** and how they differ from vector store retrievers
2. **ArxivRetriever** - Search and retrieve scholarly articles from arxiv.org
3. **WikipediaRetriever** - Access Wikipedia articles for general knowledge
4. **TavilySearchAPIRetriever** - Perform real-time internet searches
5. **Integration with RAG Chains** - Combine external retrievers with LLMs
6. **Best Practices** - When and how to use each retriever effectively

---

## Table of Contents 📚

1. [Introduction to External Retrievers](#intro)
2. [Setup & Installation](#setup)
3. [ArxivRetriever - Academic Papers](#arxiv)
4. [WikipediaRetriever - General Knowledge](#wikipedia)
5. [TavilySearchAPIRetriever - Web Search](#tavily)
6. [Integration with RAG Chains](#rag)
7. [Comparison & Use Cases](#comparison)
8. [Best Practices](#best-practices)
9. [Summary & Exercises](#summary)

---

<a id='intro'></a>
## 1. Introduction to External Index Retrievers 🔍

### What are External Index Retrievers?

**External Index Retrievers** search over external data sources (e.g., the internet, academic databases, knowledge bases) rather than your local vector store.

### Key Differences:

| Feature | Vector Store Retrievers | External Index Retrievers |
|---------|------------------------|---------------------------|
| **Data Source** | Your embedded documents | External databases/APIs |
| **Data Freshness** | Static (at indexing time) | Real-time or regularly updated |
| **Setup Required** | Embedding + Vector store | API keys (sometimes) |
| **Use Cases** | Internal documents, knowledge bases | Current events, academic research, general knowledge |
| **Cost** | Embedding cost + storage | API calls (often free tier available) |

### When to Use External Retrievers:

- ✅ You need **up-to-date information** from the internet
- ✅ You want to access **specialized databases** (e.g., academic papers)
- ✅ You need **general knowledge** without building a custom knowledge base
- ✅ You want to **augment** your local data with external sources

---

<a id='setup'></a>
## 2. Setup & Installation ⚙️

### Required Packages

All external retrievers are part of `langchain-community`. You'll also need:

```bash
pip install langchain-community
pip install arxiv           # For ArxivRetriever
pip install wikipedia       # For WikipediaRetriever
pip install tavily-python   # For TavilySearchAPIRetriever
```

### Environment Variables

For TavilySearchAPIRetriever, you'll need an API key:

```
TAVILY_API_KEY=your_api_key_here
```

Get your free API key at: https://tavily.com/

---

In [1]:
# Setup: Import required libraries
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Import LangChain components
from langchain_community.retrievers import ArxivRetriever, WikipediaRetriever, TavilySearchAPIRetriever
from langchain_core.documents import Document
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Verify versions
import langchain
print(f"✅ LangChain version: {langchain.__version__}")
print("✅ Setup complete!")

✅ LangChain version: 1.0.5
✅ Setup complete!


<a id='arxiv'></a>
## 3. ArxivRetriever - Academic Papers 📄

### 🔰 BEGINNER: What is ArxivRetriever?

**ArxivRetriever** searches [arxiv.org](https://arxiv.org), a repository of electronic preprints for research papers in:
- Physics
- Mathematics
- Computer Science
- Quantitative Biology
- Quantitative Finance
- Statistics

### Use Cases:
- 📚 Literature review for research
- 🧠 Getting latest research on AI/ML topics
- 📊 Finding papers by specific authors
- 🔬 Accessing cutting-edge research

---

### 🔰 BEGINNER: Basic ArxivRetriever Usage

In [2]:
!uv pip install arxiv

Using Python 3.11.6 environment at: simplerag
Resolved 8 packages in 618ms                                         
   Building sgmllib3k==1.0.0                                           
   Building sgmllib3k==1.0.0                                   
⠙ Preparing packages... (0/3)
   Building sgmllib3k==1.0.0-----------     0 B/79.57 KiB           
⠙ Preparing packages... (0/3)
   Building sgmllib3k==1.0.0----------- 16.00 KiB/79.57 KiB         
⠙ Preparing packages... (0/3)
   Building sgmllib3k==1.0.0----------- 32.00 KiB/79.57 KiB         
⠙ Preparing packages... (0/3)
   Building sgmllib3k==1.0.0----------- 48.00 KiB/79.57 KiB         
⠙ Preparing packages... (0/3)
   Building sgmllib3k==1.0.0m------ 64.00 KiB/79.57 KiB         
⠙ Preparing packages... (0/3)
   Building sgmllib3k==1.0.0-- 79.57 KiB/79.57 KiB         
⠙ Preparing packages... (0/3)
   Building sgmllib3k==1.0.0-- 79.57 KiB/79.57 KiB         
   Building sgmllib3k==1.0.0                                   
⠙ Preparing p

In [3]:
# Create an ArxivRetriever instance
# By default, it returns top 3 documents
arxiv_retriever = ArxivRetriever(load_max_docs=3)

# Search for papers on "large language models"
query = "large language models"
docs = arxiv_retriever.invoke(query)

print(f"📚 Found {len(docs)} papers on '{query}'\n")

# Display first paper
print("=" * 80)
print(f"Title: {docs[0].metadata.get('Title', 'N/A')}")
print(f"Authors: {docs[0].metadata.get('Authors', 'N/A')}")
print(f"Published: {docs[0].metadata.get('Published', 'N/A')}")
print(f"\nAbstract (first 500 chars):\n{docs[0].page_content[:500]}...")
print("=" * 80)
print(f"Title: {docs[1].metadata.get('Title', 'N/A')}")
print("=" * 80)
print(f"Title: {docs[2].metadata.get('Title', 'N/A')}")


📚 Found 3 papers on 'large language models'

Title: Large Language Models Lack Understanding of Character Composition of Words
Authors: Andrew Shin, Kunitake Kaneko
Published: 2024-07-23

Abstract (first 500 chars):
Large language models (LLMs) have demonstrated remarkable performances on a wide range of natural language tasks. Yet, LLMs' successes have been largely restricted to tasks concerning words, sentences, or documents, and it remains questionable how much they understand the minimal units of text, namely characters. In this paper, we examine contemporary LLMs regarding their ability to understand character composition of words, and show that most of them fail to reliably carry out even the simple t...
Title: Is Self-knowledge and Action Consistent or Not: Investigating Large Language Model's Personality
Title: Unmasking the Shadows of AI: Investigating Deceptive Capabilities in Large Language Models


### 🎓 INTERMEDIATE: Advanced ArxivRetriever Features

In [4]:
# Advanced: Retrieve more documents and explore metadata
arxiv_retriever_advanced = ArxivRetriever(
    load_max_docs=5,  # Get top 5 papers
    load_all_available_meta=True  # Load all metadata
)

# Search for papers on "transformers attention mechanism"
query = "transformers attention mechanism"
docs = arxiv_retriever_advanced.invoke(query)

print(f"📚 Retrieved {len(docs)} papers\n")

# Display metadata for all papers
for i, doc in enumerate(docs, 1):
    print(f"{i}. {doc.metadata.get('Title', 'N/A')}")
    print(f"   Authors: {doc.metadata.get('Authors', 'N/A')}")
    print(f"   Published: {doc.metadata.get('Published', 'N/A')}")
    print(f"   Entry ID: {doc.metadata.get('entry_id', 'N/A')}")
    print()

📚 Retrieved 3 papers

1. Vision Transformer with Quadrangle Attention
   Authors: Qiming Zhang, Jing Zhang, Yufei Xu, Dacheng Tao
   Published: 2023-03-27
   Entry ID: N/A

2. Déjà vu: A Contextualized Temporal Attention Mechanism for Sequential Recommendation
   Authors: Jibang Wu, Renqin Cai, Hongning Wang
   Published: 2020-01-29
   Entry ID: N/A

3. Self-Attention as Distributional Projection: A Unified Interpretation of Transformer Architecture
   Authors: Nihal Mehta
   Published: 2025-11-16
   Entry ID: N/A



### 🎓 INTERMEDIATE: Using .batch() for Multiple Queries

In [5]:
# Batch processing: Search multiple topics at once
queries = [
    "RAG retrieval augmented generation",
    "vector embeddings",
    "prompt engineering"
]

arxiv_retriever_batch = ArxivRetriever(load_max_docs=3)
batch_results = arxiv_retriever_batch.batch(queries)

print("📚 Batch Search Results:\n")
for query, docs in zip(queries, batch_results):
    print(f"Query: '{query}'")
    print(f"  → Found {len(docs)} papers")
    if docs:
        print(f"  → Top result: {docs[0].metadata.get('Title', 'N/A')}")
    print()

📚 Batch Search Results:

Query: 'RAG retrieval augmented generation'
  → Found 3 papers
  → Top result: AR-RAG: Autoregressive Retrieval Augmentation for Image Generation

Query: 'vector embeddings'
  → Found 3 papers
  → Top result: Part-of-Speech Relevance Weights for Learning Word Embeddings

Query: 'prompt engineering'
  → Found 3 papers
  → Top result: Towards Goal-oriented Prompt Engineering for Large Language Models: A Survey



### 📊 Understanding ArxivRetriever Metadata

Each document returned by ArxivRetriever contains rich metadata:

```python
{
    'Published': '2023-06-15',           # Publication date
    'Title': 'Paper Title',              # Full title
    'Authors': 'Author1, Author2',       # Comma-separated authors
    'Summary': 'Abstract text...',       # Paper abstract/summary
    'entry_id': 'http://arxiv.org/...',  # Arxiv URL
}
```

The `page_content` field contains the full abstract/summary of the paper.

---

<a id='wikipedia'></a>
## 4. WikipediaRetriever - General Knowledge 📖

### 🔰 BEGINNER: What is WikipediaRetriever?

**WikipediaRetriever** searches and retrieves content from Wikipedia, the free encyclopedia with 6+ million articles.

### Use Cases:
- 🌍 General knowledge questions
- 📚 Quick facts and definitions
- 🏛️ Historical information
- 🧑‍🔬 Biographical data
- 🗺️ Geographic information

### Important Notes:
- ⚠️ Wikipedia content is **community-edited** - verify critical information
- ✅ Great for general knowledge, not for specialized or proprietary data
- 🌐 Supports multiple languages

---

### 🔰 BEGINNER: Basic WikipediaRetriever Usage

In [6]:
!uv pip install wikipedia

Using Python 3.11.6 environment at: simplerag
Resolved 9 packages in 560ms                                         
   Building wikipedia==1.4.0                                           
      Built wikipedia==1.4.0                                   
Prepared 1 package in 137ms                                              
Installed 1 package in 1ms                                  
 + wikipedia==1.4.0


In [7]:
# Create a WikipediaRetriever instance
# By default, it returns top 3 documents
wiki_retriever = WikipediaRetriever(top_k_results=2)

# Search for information on "Python programming language"
query = "Python programming language"
docs = wiki_retriever.invoke(query)

print(f"📖 Found {len(docs)} Wikipedia articles on '{query}'\n")

# Display first result
print("=" * 80)
print(f"Title: {docs[0].metadata.get('title', 'N/A')}")
print(f"Source: {docs[0].metadata.get('source', 'N/A')}")
print(f"\nContent (first 600 chars):\n{docs[0].page_content[:600]}...")
print("=" * 80)

📖 Found 2 Wikipedia articles on 'Python programming language'

Title: Python (programming language)
Source: https://en.wikipedia.org/wiki/Python_(programming_language)

Content (first 600 chars):
Python is a high-level, general-purpose programming language. Its design philosophy emphasizes code readability with the use of significant indentation. Python is dynamically type-checked and garbage-collected. It supports multiple programming paradigms, including structured (particularly procedural), object-oriented and functional programming.
Guido van Rossum began working on Python in the late 1980s as a successor to the ABC programming language. Python 3.0, released in 2008, was a major revision and not completely backward-compatible with earlier versions. Beginning with Python 3.5, capabi...


### 🎓 INTERMEDIATE: Advanced WikipediaRetriever Features

In [8]:
# Advanced: Control number of results and document length
wiki_retriever_advanced = WikipediaRetriever(
    top_k_results=3,        # Get top 3 results
    doc_content_chars_max=1000  # Limit content to 1000 characters per doc
)

# Search for "Machine Learning"
query = "Machine Learning"
docs = wiki_retriever_advanced.invoke(query)

print(f"📖 Retrieved {len(docs)} Wikipedia articles\n")

# Display all results
for i, doc in enumerate(docs, 1):
    print(f"{i}. Title: {doc.metadata.get('title', 'N/A')}")
    print(f"   Summary: {doc.metadata.get('summary', 'N/A')[:150]}...")
    print(f"   Content length: {len(doc.page_content)} characters")
    print()

📖 Retrieved 3 Wikipedia articles

1. Title: Machine learning
   Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn...
   Content length: 1000 characters

2. Title: Neural network (machine learning)
   Summary: In machine learning, a neural network or neural net (NN), also called artificial neural network (ANN), is a computational model inspired by the struct...
   Content length: 1000 characters

3. Title: Quantum machine learning
   Summary: Quantum machine learning (QML), pioneered by Ventura and Martinez  and by Trugenberger in the late 1990s and early 2000s, is the study of quantum algo...
   Content length: 1000 characters



### 🎓 INTERMEDIATE: Multilingual Support

In [9]:
# Search in different languages
# Default is English ('en'), but you can specify other languages

# Example: Search in Spanish
wiki_retriever_es = WikipediaRetriever(
    top_k_results=1,
    lang="es"  # Spanish Wikipedia
)

query = "Inteligencia Artificial"
docs = wiki_retriever_es.invoke(query)

print(f"🌐 Search in Spanish Wikipedia: '{query}'\n")
print(f"Title: {docs[0].metadata.get('title', 'N/A')}")
print(f"Content preview:\n{docs[0].page_content[:400]}...")

🌐 Search in Spanish Wikipedia: 'Inteligencia Artificial'

Title: Inteligencia artificial
Content preview:
La inteligencia artificial, abreviado como IA, en el contexto de las ciencias de la computación, es una disciplina y un conjunto de capacidades cognoscitivas e intelectuales expresadas por sistemas informáticos o combinaciones de algoritmos cuyo propósito es la creación de máquinas que imiten la inteligencia humana.
Estas tecnologías permiten que las máquinas aprendan de la experiencia, se adapten...


### 🎓 INTERMEDIATE: Batch Processing with WikipediaRetriever

In [10]:
# Batch search for multiple topics
queries = [
    "Albert Einstein",
    "Quantum Computing",
    "Neural Networks"
]

wiki_retriever_batch = WikipediaRetriever(top_k_results=1, doc_content_chars_max=500)
batch_results = wiki_retriever_batch.batch(queries)

print("📖 Batch Wikipedia Search Results:\n")
for query, docs in zip(queries, batch_results):
    print(f"Query: '{query}'")
    if docs:
        print(f"  → Title: {docs[0].metadata.get('title', 'N/A')}")
        print(f"  → Summary: {docs[0].page_content[:200]}...")
    print()

📖 Batch Wikipedia Search Results:

Query: 'Albert Einstein'
  → Title: Albert Einstein
  → Summary: Albert Einstein (14 March 1879 – 18 April 1955) was a German-born theoretical physicist best known for developing the theory of relativity. Einstein also made important contributions to quantum theory...

Query: 'Quantum Computing'
  → Title: Quantum computing
  → Summary: A quantum computer is a (real or theoretical) computer that exploits superposed and entangled states, and the intrinsically non-deterministic outcomes of quantum measurements, as features of its compu...

Query: 'Neural Networks'
  → Title: Neural network (machine learning)
  → Summary: In machine learning, a neural network or neural net (NN), also called artificial neural network (ANN), is a computational model inspired by the structure and functions of biological neural networks.
A...



### 📊 Understanding WikipediaRetriever Metadata

Each document returned by WikipediaRetriever contains:

```python
{
    'title': 'Article Title',           # Wikipedia article title
    'summary': 'Brief summary...',       # Short summary (if available)
    'source': 'https://en.wikipedia...', # Full Wikipedia URL
}
```

The `page_content` field contains the article text (up to `doc_content_chars_max` characters).

---

<a id='tavily'></a>
## 5. TavilySearchAPIRetriever - Web Search 🔍

### 🔰 BEGINNER: What is TavilySearchAPIRetriever?

**TavilySearchAPIRetriever** performs **real-time internet searches** using the Tavily Search API, optimized for AI applications.

### Key Features:
- 🌐 **Real-time web search** - Get the latest information from the internet
- 🎯 **AI-optimized** - Returns clean, relevant content for LLMs
- 🔒 **Source attribution** - Includes URLs and metadata
- ⚡ **Fast & reliable** - Built specifically for AI use cases

### Use Cases:
- 📰 Current events and news
- 💹 Stock prices and market data
- 🌦️ Weather information
- 🏢 Company information
- 🔧 Technical documentation and tutorials

### Getting Started:
1. Sign up at https://tavily.com/ (free tier available)
2. Get your API key
3. Add to `.env` file: `TAVILY_API_KEY=your_api_key`

---

### 🔰 BEGINNER: Basic TavilySearchAPIRetriever Usage

In [11]:
!uv pip install tavily-python

Using Python 3.11.6 environment at: simplerag
Resolved 14 packages in 254ms                                        
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)-------------------     0 B/15.12 KiB           
⠙ Preparing packages... (0/1)---------- 14.85 KiB/15.12 KiB         
⠙ Preparing packages... (0/1)---------- 15.12 KiB/15.12 KiB         
Prepared 1 package in 28ms                                                        
Installed 1 package in 2ms13                                
 + tavily-python==0.7.13


In [12]:
# Create a TavilySearchAPIRetriever instance
# Make sure TAVILY_API_KEY is set in your .env file

tavily_retriever = TavilySearchAPIRetriever(k=3)  # Return top 3 results

# Search for "latest developments in artificial intelligence 2024"
query = "latest developments in artificial intelligence 2024"
docs = tavily_retriever.invoke(query)

print(f"🔍 Found {len(docs)} web results for '{query}'\n")

# Display first result
print("=" * 80)
print(f"Source: {docs[0].metadata.get('source', 'N/A')}")
print(f"\nContent (first 500 chars):\n{docs[0].page_content[:500]}...")
print("=" * 80)

🔍 Found 3 web results for 'latest developments in artificial intelligence 2024'

Source: https://www.launchconsulting.com/posts/the-future-of-business-ai-innovations-to-watch-in-2024

Content (first 500 chars):
Key AI trends for 2024 include generative AI beyond chatbots, small language models, multi-modal experiences, AI for professionals, and evolving legislation....


### 🎓 INTERMEDIATE: Advanced TavilySearchAPIRetriever Features

In [13]:
# Advanced: Control search depth and domain filtering
from langchain_community.retrievers import TavilySearchAPIRetriever

# Advanced configuration
tavily_retriever_advanced = TavilySearchAPIRetriever(
    k=5,  # Return top 5 results
    # search_depth="advanced",  # "basic" or "advanced" (more thorough)
    # include_domains=["github.com", "stackoverflow.com"],  # Filter to specific domains
    # exclude_domains=["example.com"]  # Exclude specific domains
)

# Search for "LangChain tutorials"
query = "LangChain tutorials"
docs = tavily_retriever_advanced.invoke(query)

print(f"🔍 Retrieved {len(docs)} web results\n")

# Display all results with sources
for i, doc in enumerate(docs, 1):
    print(f"{i}. Source: {doc.metadata.get('source', 'N/A')}")
    print(f"   Content preview: {doc.page_content[:200]}...")
    print()

🔍 Retrieved 5 web results

1. Source: https://www.datacamp.com/tutorial/how-to-build-llm-applications-with-langchain
   Content preview: This tutorial is perfect for you. Here, we explore LangChain - An open-source Python framework for building applications based on Large Language Models such as...

2. Source: https://medium.com/data-science-in-your-pocket/langchain-tutorials-for-newbies-945319df04e2
   Content preview: ## LangChain in your Pocket: Beginner's Guide to Building Generative AI Applications using LLMs ### LangChain in your Pocket: Beginner's Guide to Building Generative AI Applications using LLMs eBook :...

3. Source: https://docs.langchain.com/oss/python/learn
   Content preview: [LangChain](/oss/python/langchain/overview)[LangGraph](/oss/python/langgraph/overview)[Deep Agents](/oss/python/deepagents/overview)[Integrations](/oss/python/integrations/providers/overview)[Learn](/...

4. Source: https://github.com/gkamradt/langchain-tutorials
   Content preview: 2. LangChain

### 🎓 INTERMEDIATE: Real-Time Information Retrieval

In [14]:
# Example: Get current information (news, weather, stock prices, etc.)
from datetime import datetime

current_date = datetime.now().strftime("%B %d, %Y")

# Real-time queries
queries = [
    f"latest AI news {current_date}",
    "current weather in San Francisco",
    "NVIDIA stock price today"
]

tavily_realtime = TavilySearchAPIRetriever(k=2)

print(f"🕐 Real-Time Information (as of {current_date}):\n")

for query in queries:
    docs = tavily_realtime.invoke(query)
    print(f"Query: '{query}'")
    if docs:
        print(f"  → {docs[0].page_content[:250]}...")
        print(f"  → Source: {docs[0].metadata.get('source', 'N/A')}")
    print()

🕐 Real-Time Information (as of November 24, 2025):

Query: 'latest AI news November 24, 2025'
  → Nov 24, 2025, 03:45am EST. Contraction Before Expansion: How AI is Poised Spark Tech Services Growth. getty. Over the last two years, something unusual has...
  → Source: https://www.forbes.com/sites/peterbendorsamuel/2025/11/24/contraction-before-expansion-how-ai-is-poised-spark-tech-services-growth/

Query: 'current weather in San Francisco'
  → {'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1763982148, 'localtime': '2025-11-24 03:02'}, 'current': {'last_...
  → Source: https://www.weatherapi.com/

Query: 'NVIDIA stock price today'
  → The NVIDIA Corporation Common Stock (NVDA) stock price today is $180.03, reflecting a -0.49% move since the market opened. The company's market capitalization...
  → Source: https://www.kraken.com/stocks/nvda



### 📊 Understanding TavilySearchAPIRetriever Metadata

Each document returned by TavilySearchAPIRetriever contains:

```python
{
    'source': 'https://example.com/...',  # Source URL
    'score': 0.95,                         # Relevance score (0-1)
    'title': 'Page Title',                 # Web page title (if available)
}
```

The `page_content` field contains the extracted text content from the web page.

---

<a id='rag'></a>
## 6. Integration with RAG Chains 🔗

Now let's combine external retrievers with LLMs to build powerful **Retrieval-Augmented Generation (RAG)** systems!

### 🔰 BEGINNER: Simple QA Chain with External Retriever

In [15]:
# Build a simple RAG chain using WikipediaRetriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Initialize components
wiki_retriever = WikipediaRetriever(top_k_results=2, doc_content_chars_max=2000)
llm = ChatOpenAI(model="gpt-5-nano", temperature=0)

# Create prompt template
template = """Answer the question based on the following context from Wikipedia:

Context:
{context}

Question: {question}

Answer:"""

prompt = ChatPromptTemplate.from_template(template)

# Helper function to format documents
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Build the RAG chain using LCEL
rag_chain = (
    {"context": wiki_retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Ask a question
question = "What is quantum computing and how does it work?"
answer = rag_chain.invoke(question)

print(f"Question: {question}\n")
print(f"Answer: {answer}")

Question: What is quantum computing and how does it work?

Answer: Quantum computing is a form of computing that uses quantum bits, or qubits, which can be in a superposition of 0 and 1 and can become entangled with one another. A quantum computer works by preparing qubits, applying quantum operations (gates) to them, and then measuring the qubits. Measurements are probabilistic: you get 0 or 1 according to quantum probabilities. By carefully manipulating the qubits, wave interference can amplify the probability of the desired outcome.

Because a system of qubits can represent many possible states at once, a quantum computer can explore a large space of possibilities in parallel and sample from it, though it still has computational constraints. Algorithms are designed to construct procedures that increase the likelihood of the correct result when measured.

As of now, quantum computers are not yet practical for general use; current hardware is largely experimental and suitable only for

### 🎓 INTERMEDIATE: Multi-Source RAG Chain

In [16]:
# Advanced: Combine multiple retrievers for comprehensive answers
from langchain_core.runnables import RunnableParallel

# Initialize multiple retrievers
arxiv_retriever = ArxivRetriever(load_max_docs=2)
wiki_retriever = WikipediaRetriever(top_k_results=2, doc_content_chars_max=1500)

# Function to combine results from multiple retrievers
def multi_retriever(query):
    """Retrieve from multiple sources and combine results."""
    arxiv_docs = arxiv_retriever.invoke(query)
    wiki_docs = wiki_retriever.invoke(query)
    
    # Combine and format
    all_docs = []
    
    if arxiv_docs:
        all_docs.append("=== Academic Papers (ArXiv) ===")
        all_docs.extend([doc.page_content[:500] for doc in arxiv_docs])
    
    if wiki_docs:
        all_docs.append("\n=== General Knowledge (Wikipedia) ===")
        all_docs.extend([doc.page_content[:500] for doc in wiki_docs])
    
    return "\n\n".join(all_docs)

# Create multi-source RAG chain
multi_source_template = """Answer the question using information from multiple sources below:

{context}

Question: {question}

Provide a comprehensive answer that synthesizes information from both academic and general sources:"""

multi_prompt = ChatPromptTemplate.from_template(multi_source_template)

multi_rag_chain = (
    {"context": multi_retriever, "question": RunnablePassthrough()}
    | multi_prompt
    | llm
    | StrOutputParser()
)

# Ask a question
question = "What are transformers in machine learning?"
answer = multi_rag_chain.invoke(question)

print(f"Question: {question}\n")
print(f"Answer (from multiple sources):\n{answer}")

Question: What are transformers in machine learning?

Answer (from multiple sources):
A transformer is a type of neural network architecture designed to model relationships in sequences using attention mechanisms, rather than relying on recurrence (like RNNs) or fixed convolutional windows. It is built around the idea of computing how much each part of a input sequence should influence every other part, and it does this using multi-head self-attention over token representations.

What “transformers” do and what they rely on
- Tokens and embeddings: The input data are first converted into tokens, and each token is mapped to a vector through an embedding table. This turns discrete data (like words) into continuous vectors that the network can manipulate. (Wikipedia description)
- Contextualization with multi-head attention: At each layer, every token is contextualized with respect to the rest of the tokens in the input via multi-head attention. This means the model learns which other tok

### 🎓 INTERMEDIATE: Real-Time RAG with TavilySearchAPIRetriever

In [17]:
# Build a RAG chain that uses real-time web search
tavily_retriever = TavilySearchAPIRetriever(k=3)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# Create prompt for real-time information
realtime_template = """Based on the latest information from the web:

{context}

Question: {question}

Provide an up-to-date answer having atleast 500 words with source attribution:"""

realtime_prompt = ChatPromptTemplate.from_template(realtime_template)

# Build real-time RAG chain
realtime_rag_chain = (
    {"context": tavily_retriever | format_docs, "question": RunnablePassthrough()}
    | realtime_prompt
    | llm
    | StrOutputParser()
)

# Ask a current events question
question = "What are the latest developments in AI regulation?"
answer = realtime_rag_chain.invoke(question)

print(f"Question: {question}\n")
print(f"Answer (from real-time web search):\n{answer}")

Question: What are the latest developments in AI regulation?

Answer (from real-time web search):
As of the latest developments in AI regulation, countries around the world are increasingly recognizing the need for comprehensive laws and guidelines to govern the use of artificial intelligence. The European Union's AI Act, which was proposed in April 2021 and is set to come into effect on August 1, 2024, is one of the most significant pieces of AI regulation to date. This regulation aims to establish harmonized rules for AI across all 27 EU member states, addressing issues such as transparency, accountability, and data governance.

One of the key provisions of the EU AI Act is the creation of a regulatory framework for high-risk AI applications. These are defined as AI systems that pose significant risks to the health, safety, or fundamental rights of individuals. Examples of high-risk AI applications include autonomous vehicles, facial recognition technology, and predictive policing sy

---

<a id='comparison'></a>
## 7. Comparison & Use Cases 📊

### Retriever Comparison Table

| Feature | ArxivRetriever | WikipediaRetriever | TavilySearchAPIRetriever |
|---------|----------------|-------------------|-------------------------|
| **Data Source** | Academic papers (arxiv.org) | Wikipedia articles | Real-time web search |
| **API Key Required** | ❌ No | ❌ No | ✅ Yes (free tier) |
| **Data Freshness** | Recent research | Regularly updated | Real-time |
| **Best For** | Academic research, ML papers | General knowledge, definitions | Current events, news |
| **Content Type** | Research papers, abstracts | Encyclopedia articles | Web pages, news |
| **Default Results** | 3 papers | 3 articles | 5 results |
| **Multilingual** | ❌ No | ✅ Yes (300+ languages) | ✅ Yes |
| **Metadata** | Title, Authors, Published date | Title, Summary, URL | Source URL, Score |
| **Rate Limits** | Moderate | Moderate | API-dependent |
| **Cost** | 🆓 Free | 🆓 Free | 🆓 Free tier + paid |

---

### When to Use Each Retriever

#### ✅ Use **ArxivRetriever** when:
- You need peer-reviewed academic research
- You're building an AI/ML research assistant
- You want the latest scientific papers
- You need citations and author information

#### ✅ Use **WikipediaRetriever** when:
- You need general knowledge and definitions
- You want historical or biographical information
- You're building an educational chatbot
- You need multilingual support
- You want reliable, community-edited content

#### ✅ Use **TavilySearchAPIRetriever** when:
- You need real-time, up-to-date information
- You're answering current events questions
- You want to search the broader internet
- You need to filter by specific domains
- Your use case requires the latest data

---

### Combining Retrievers (Hybrid Approach)

For the most comprehensive RAG system:

```python
# Pseudo-code for hybrid retrieval
if query_type == "academic":
    use ArxivRetriever
elif query_type == "general_knowledge":
    use WikipediaRetriever
elif query_type == "current_events":
    use TavilySearchAPIRetriever
else:
    # Use multiple retrievers and combine results
    combine(ArxivRetriever, WikipediaRetriever, TavilySearchAPIRetriever)
```

---

<a id='best-practices'></a>
## 8. Best Practices 💡

### General Best Practices

#### 1. **Handle Errors Gracefully**

```python
try:
    docs = retriever.invoke(query)
except Exception as e:
    print(f"Error retrieving documents: {e}")
    docs = []  # Fallback to empty list
```

#### 2. **Set Appropriate Limits**

```python
# Don't retrieve too many documents (costs, latency)
arxiv_retriever = ArxivRetriever(load_max_docs=3)  # ✅ Good
arxiv_retriever = ArxivRetriever(load_max_docs=100)  # ❌ Too many
```

#### 3. **Cache Results for Repeated Queries**

```python
# Use a simple cache to avoid redundant API calls
from functools import lru_cache

@lru_cache(maxsize=100)
def cached_search(query: str):
    return retriever.invoke(query)
```

#### 4. **Verify Source Attribution**

```python
# Always include sources in your responses
for doc in docs:
    print(f"Source: {doc.metadata.get('source', 'N/A')}")
```

#### 5. **Combine with Vector Store Retrievers**

```python
# Use external retrievers for general knowledge
# Use vector stores for your proprietary data
def hybrid_retrieve(query):
    external_docs = wiki_retriever.invoke(query)
    internal_docs = vector_store.similarity_search(query)
    return external_docs + internal_docs
```

---

### Retriever-Specific Best Practices

#### ArxivRetriever:
- ✅ Use specific search terms (e.g., "BERT transformers" vs "AI")
- ✅ Limit results to 3-5 papers for LLM context
- ✅ Extract metadata for citations
- ❌ Don't use for non-academic queries

#### WikipediaRetriever:
- ✅ Use for general knowledge, not specialized topics
- ✅ Set `doc_content_chars_max` to avoid huge documents
- ✅ Verify information for critical use cases
- ❌ Don't rely on Wikipedia for real-time information

#### TavilySearchAPIRetriever:
- ✅ Monitor API usage (rate limits, costs)
- ✅ Use for time-sensitive queries
- ✅ Filter by domain for specific sources
- ❌ Don't use for queries that don't need real-time data

---

### Performance Tips

1. **Use `.batch()` for multiple queries**
   ```python
   # ✅ Efficient
   results = retriever.batch([q1, q2, q3])
   
   # ❌ Inefficient
   results = [retriever.invoke(q) for q in [q1, q2, q3]]
   ```

2. **Limit document length for LLM context**
   ```python
   # Truncate long documents to fit LLM context window
   docs = [Document(page_content=doc.page_content[:2000], metadata=doc.metadata) 
           for doc in raw_docs]
   ```

3. **Use async methods for concurrent retrieval** (if supported)
   ```python
   # For async-compatible retrievers
   import asyncio
   docs = await retriever.ainvoke(query)
   ```

---

<a id='summary'></a>
## 9. Summary & Exercises 📝

### 🎯 What You Learned

In this notebook, you learned:

✅ **External Index Retrievers** - Search over external data sources (internet, databases)

✅ **ArxivRetriever** - Retrieve academic papers from arxiv.org
   - Use cases: Research, ML papers, citations
   - Methods: `.invoke()`, `.batch()`
   - Metadata: Title, Authors, Published date

✅ **WikipediaRetriever** - Access Wikipedia articles
   - Use cases: General knowledge, definitions, history
   - Features: Multilingual support, customizable length
   - Metadata: Title, Summary, Source URL

✅ **TavilySearchAPIRetriever** - Real-time web search
   - Use cases: Current events, news, real-time data
   - Features: Domain filtering, search depth control
   - Metadata: Source URL, Relevance score

✅ **RAG Integration** - Combined external retrievers with LLMs
   - Built simple QA chains
   - Created multi-source RAG systems
   - Implemented real-time information retrieval

✅ **Best Practices** - Error handling, caching, source attribution

---

### 💪 Practice Exercises

#### Exercise 1: Academic Research Assistant (🔰 Beginner)
Create a RAG chain that:
- Uses `ArxivRetriever` to find papers on "deep learning"
- Extracts the top 3 paper titles and authors
- Summarizes each paper's abstract using an LLM

#### Exercise 2: Wikipedia Fact Checker (🔰 Beginner)
Build a system that:
- Takes a statement as input (e.g., "Python was created in 1991")
- Uses `WikipediaRetriever` to search for relevant articles
- Uses an LLM to verify if the statement is accurate

#### Exercise 3: Multi-Source News Aggregator (🎓 Intermediate)
Create a RAG chain that:
- Uses `TavilySearchAPIRetriever` to get latest AI news
- Uses `WikipediaRetriever` to get background on AI topics
- Combines both sources to provide a comprehensive news summary

#### Exercise 4: Hybrid Retrieval System (🎓 Intermediate)
Build a system that:
- Classifies queries into "academic", "general", or "current_events"
- Routes to the appropriate retriever based on query type
- Returns results from the most relevant source

#### Exercise 5: Multilingual Knowledge Base (🚀 Advanced)
Create a system that:
- Detects the language of the user's query
- Uses `WikipediaRetriever` with the appropriate language setting
- Returns answers in the user's language

---

### 🔗 Next Steps

- **Notebook 09**: Advanced Retrieval Techniques (Hybrid Search, Re-ranking)
- **Notebook 10**: Production RAG Systems (Caching, Monitoring, Scaling)
- **LangChain Documentation**: https://python.langchain.com/docs/integrations/retrievers/

---

### 📚 Additional Resources

- **ArXiv**: https://arxiv.org/
- **Wikipedia API**: https://www.mediawiki.org/wiki/API:Main_page
- **Tavily API**: https://tavily.com/
- **LangChain Retrievers**: https://python.langchain.com/docs/modules/data_connection/retrievers/

---

**Congratulations!** 🎉 You've mastered external index retrievers in LangChain!

You can now build RAG systems that access:
- 📄 Academic research (ArXiv)
- 📖 General knowledge (Wikipedia)
- 🌐 Real-time web data (Tavily)

Keep experimenting and building amazing AI applications! 🚀